In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model

# Example: Handling multiple shots separately
# Assume that `shots_data` is a list of tuples, each containing (velocity, round_info, time_series_data)
# time_series_data would be a 3D array of shape (timesteps, num_particles, features)

# Define the model architecture for one shot
def create_shot_model(timesteps, num_particles, feature_dim):
    # Input layers
    velocity_input = Input(shape=(1,), name='velocity_input')
    round_input = Input(shape=(1,), name='round_input')
    particle_input = Input(shape=(timesteps, num_particles, feature_dim), name='particle_input')

    # Flatten the round input if using embeddings
    round_emb = tf.keras.layers.Embedding(input_dim=10, output_dim=4)(round_input)  # Example for 10 rounds
    round_emb = tf.keras.layers.Flatten()(round_emb)

    # LSTM to process the particle data
    lstm_out = TimeDistributed(LSTM(64, return_sequences=True))(particle_input)

    # Repeat velocity and round information across time steps and particles
    velocity_expanded = tf.keras.layers.RepeatVector(timesteps * num_particles)(velocity_input)
    velocity_expanded = tf.keras.layers.Reshape((timesteps, num_particles, 1))(velocity_expanded)

    round_expanded = tf.keras.layers.RepeatVector(timesteps * num_particles)(round_emb)
    round_expanded = tf.keras.layers.Reshape((timesteps, num_particles, -1))(round_expanded)

    # Combine the features
    combined_features = Concatenate()([lstm_out, velocity_expanded, round_expanded])

    # Output layer - Binary classification for drop status
    drop_status = TimeDistributed(Dense(1, activation='sigmoid'))(combined_features)

    # Create model
    model = Model(inputs=[velocity_input, round_input, particle_input], outputs=drop_status)
    return model

# Assume that the data is preprocessed and segmented into shots
# For each shot, you would then train the model separately
for shot in shots_data:
    velocity, round_info, time_series_data = shot
    timesteps, num_particles, feature_dim = time_series_data.shape

    # Create a new model for each shot or share the same model across shots
    model = create_shot_model(timesteps, num_particles, feature_dim)
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model on this specific shot data
    model.fit([velocity, round_info, time_series_data], drop_status_labels, epochs=10)

    # Optionally, save the model for this specific shot
    model.save(f"model_shot_{round_info}.h5")
